In [2]:
import numpy as np
import copy
import heapq

In [3]:
def print_succ(state):
    states = successors(state)
    for s in states: 
            print(s, "h= " + str(heuristic(s)))

In [4]:
def successors(state):
    """
    Returns: a list of successors to the current state
    """
    board = np.array(state).reshape(3,3)
    states = []
    
    space_coordinate = ()
    for row in range(3):
        for col in range(3):
            if board[row][col] == 0:
                space_coordinate = (row, col)
        else:
            continue
        break
    
    row = space_coordinate[0]
    col = space_coordinate[1]
    
    if row == 0:
        states.append(list(copy_board(board, row, col, 1, 0).flatten()))
    elif row == 2: 
        states.append(list(copy_board(board, row, col, -1, 0).flatten()))
    else:
        states.append(list(copy_board(board, row, col, 1, 0).flatten()))
        states.append(list(copy_board(board, row, col, -1, 0).flatten()))
    
    if col == 0:
        states.append(list(copy_board(board, row, col, 0, 1).flatten()))
    elif col == 2:
        states.append(list(copy_board(board, row, col, 0, -1).flatten()))
    else:
        states.append(list(copy_board(board, row, col, 0, 1).flatten()))
        states.append(list(copy_board(board, row, col, 0, -1).flatten()))
    return sorted(states)


In [5]:
def copy_board(board, row, col, move_row, move_col):
    """
    Copies the board reference and makes changes to find a successor
    """
    new_board = copy.deepcopy(board)
    new_board[row][col] = new_board[row+move_row][col+move_col]
    new_board[row+move_row][col+move_col] = 0
    return new_board

In [6]:
def heuristic(state):
    """
    Helper function for print_succ
    Returns: value of heuristic function of current state 
    """
    h = 0
    goal = [(0,0), (0,1), (0,2), (1,0), (1,1), (1,2), (2,0), (2,1)]
    board = np.array(state).reshape(3,3)
    for row in range(3):
        for col in range(3):
            label = board[row][col]
            if label == 0:
                continue
            h += abs(goal[label-1][0] - row) + abs(goal[label-1][1] - col)
    return h

In [7]:
def solve(state):
    
    goal = [1, 2, 3, 4, 5, 6, 7, 8, 0]
    pq = []
    path = []
    list_parents = []
    heapq.heappush(pq, (heuristic(state), state, (0, heuristic(state), -1)))
    while(len(pq) != 0):
        n = heapq.heappop(pq)
        path.append(n)
        if n[1] == goal:
            break
        for successor in successors(n[1]):
            list_parents.append((successor, n[1]))
            parent_index = n[2][2] + 1
            g =  n[2][0] + 1
            h = heuristic(successor)
            n_next = (g + h, successor, (g, h, parent_index))
            in_path = False
            in_pq = False            
                    
            for s in pq:
                if s[1] == successor:
                    in_pq = True
                    break

            for s in path:
                if s[1] == successor:
                    in_path = True
                    break
            
            if (not in_path) and (not in_pq):
                heapq.heappush(pq, n_next)
    
    next_node = path[-1]
    p = []
    for i, curr in reversed(list(enumerate(path))):
        if next_node != curr:
            continue
        if curr[2][2] == -1:
            p.append(curr)
            break
            
        parent_found = False
        
        curr_parent = []
        for pair in reversed(list_parents):
            if pair[0] == curr[1]:
                if pair[1] == path[i-1][1]:
                    next_node = path[i - 1]
                    parent_found = True
                    break
                else:
                    curr_parent.append(pair[1])
                    
        if not parent_found:
            n = 2
            while(path[i-n][1] not in curr_parent):
                n = n + 1
            next_node = path[i-n]
        p.append(curr)
            
            
    for board in reversed(p):
        print("{} h={} moves: {}".format(board[1], board[2][1], board[2][0]))

In [172]:
solve([1,2,3,0,4,6,7,5,8])

[1, 2, 3, 0, 4, 6, 7, 5, 8] h=3 moves: 0
[1, 2, 3, 4, 0, 6, 7, 5, 8] h=2 moves: 1
[1, 2, 3, 4, 5, 6, 7, 0, 8] h=1 moves: 2
[1, 2, 3, 4, 5, 6, 7, 8, 0] h=0 moves: 3


In [173]:
solve([1,2,3,4,5,6,7,0,8])

[1, 2, 3, 4, 5, 6, 7, 0, 8] h=1 moves: 0
[1, 2, 3, 4, 5, 6, 7, 8, 0] h=0 moves: 1


In [174]:
solve([4,3,8,5,1,6,7,2,0])

[4, 3, 8, 5, 1, 6, 7, 2, 0] h=10 moves: 0
[4, 3, 8, 5, 1, 0, 7, 2, 6] h=11 moves: 1
[4, 3, 0, 5, 1, 8, 7, 2, 6] h=10 moves: 2
[4, 0, 3, 5, 1, 8, 7, 2, 6] h=9 moves: 3
[4, 1, 3, 5, 0, 8, 7, 2, 6] h=8 moves: 4
[4, 1, 3, 5, 8, 0, 7, 2, 6] h=7 moves: 5
[4, 1, 3, 5, 8, 6, 7, 2, 0] h=6 moves: 6
[4, 1, 3, 5, 8, 6, 7, 0, 2] h=7 moves: 7
[4, 1, 3, 5, 0, 6, 7, 8, 2] h=6 moves: 8
[4, 1, 3, 0, 5, 6, 7, 8, 2] h=5 moves: 9
[0, 1, 3, 4, 5, 6, 7, 8, 2] h=4 moves: 10
[1, 0, 3, 4, 5, 6, 7, 8, 2] h=3 moves: 11
[1, 3, 0, 4, 5, 6, 7, 8, 2] h=4 moves: 12
[1, 3, 6, 4, 5, 0, 7, 8, 2] h=5 moves: 13
[1, 3, 6, 4, 5, 2, 7, 8, 0] h=4 moves: 14
[1, 3, 6, 4, 5, 2, 7, 0, 8] h=5 moves: 15
[1, 3, 6, 4, 0, 2, 7, 5, 8] h=6 moves: 16
[1, 3, 6, 4, 2, 0, 7, 5, 8] h=5 moves: 17
[1, 3, 0, 4, 2, 6, 7, 5, 8] h=4 moves: 18
[1, 0, 3, 4, 2, 6, 7, 5, 8] h=3 moves: 19
[1, 2, 3, 4, 0, 6, 7, 5, 8] h=2 moves: 20
[1, 2, 3, 4, 5, 6, 7, 0, 8] h=1 moves: 21
[1, 2, 3, 4, 5, 6, 7, 8, 0] h=0 moves: 22


In [175]:
solve([8,6,7,2,5,4,3,0,1])

[8, 6, 7, 2, 5, 4, 3, 0, 1] h=21 moves: 0
[8, 6, 7, 2, 5, 4, 3, 1, 0] h=20 moves: 1
[8, 6, 7, 2, 5, 0, 3, 1, 4] h=21 moves: 2
[8, 6, 0, 2, 5, 7, 3, 1, 4] h=20 moves: 3
[8, 0, 6, 2, 5, 7, 3, 1, 4] h=19 moves: 4
[0, 8, 6, 2, 5, 7, 3, 1, 4] h=18 moves: 5
[2, 8, 6, 0, 5, 7, 3, 1, 4] h=17 moves: 6
[2, 8, 6, 5, 0, 7, 3, 1, 4] h=18 moves: 7
[2, 0, 6, 5, 8, 7, 3, 1, 4] h=17 moves: 8
[0, 2, 6, 5, 8, 7, 3, 1, 4] h=16 moves: 9
[5, 2, 6, 0, 8, 7, 3, 1, 4] h=17 moves: 10
[5, 2, 6, 8, 0, 7, 3, 1, 4] h=18 moves: 9
[5, 2, 6, 8, 1, 7, 3, 0, 4] h=17 moves: 10
[5, 2, 6, 8, 1, 7, 0, 3, 4] h=16 moves: 11
[5, 2, 6, 0, 1, 7, 8, 3, 4] h=15 moves: 12
[5, 2, 6, 1, 0, 7, 8, 3, 4] h=14 moves: 13
[5, 2, 6, 1, 7, 0, 8, 3, 4] h=13 moves: 14
[5, 2, 0, 1, 7, 6, 8, 3, 4] h=12 moves: 15
[5, 0, 2, 1, 7, 6, 8, 3, 4] h=13 moves: 16
[0, 5, 2, 1, 7, 6, 8, 3, 4] h=12 moves: 17
[1, 5, 2, 0, 7, 6, 8, 3, 4] h=11 moves: 18
[1, 5, 2, 7, 0, 6, 8, 3, 4] h=10 moves: 19
[1, 5, 2, 7, 3, 6, 8, 0, 4] h=9 moves: 20
[1, 5, 2, 7, 3, 6, 8, 4

In [177]:
solve([6,5,0,8,2,4,3,1,7])

[6, 5, 0, 8, 2, 4, 3, 1, 7] h=18 moves: 0
[6, 0, 5, 8, 2, 4, 3, 1, 7] h=19 moves: 1
[6, 2, 5, 8, 0, 4, 3, 1, 7] h=18 moves: 2
[6, 2, 5, 0, 8, 4, 3, 1, 7] h=17 moves: 3
[6, 2, 5, 3, 8, 4, 0, 1, 7] h=16 moves: 4
[6, 2, 5, 3, 8, 4, 1, 0, 7] h=15 moves: 5
[6, 2, 5, 3, 8, 4, 1, 7, 0] h=14 moves: 6
[6, 2, 5, 3, 8, 0, 1, 7, 4] h=15 moves: 7
[6, 2, 5, 3, 0, 8, 1, 7, 4] h=16 moves: 8
[6, 2, 5, 0, 3, 8, 1, 7, 4] h=15 moves: 9
[0, 2, 5, 6, 3, 8, 1, 7, 4] h=14 moves: 10
[2, 0, 5, 6, 3, 8, 1, 7, 4] h=15 moves: 11
[2, 3, 5, 6, 0, 8, 1, 7, 4] h=14 moves: 12
[2, 3, 5, 0, 6, 8, 1, 7, 4] h=13 moves: 13
[2, 3, 5, 1, 6, 8, 0, 7, 4] h=12 moves: 14
[2, 3, 5, 1, 6, 8, 7, 0, 4] h=11 moves: 15
[2, 3, 5, 1, 6, 8, 7, 4, 0] h=10 moves: 16
[2, 3, 5, 1, 6, 0, 7, 4, 8] h=9 moves: 17
[2, 3, 5, 1, 0, 6, 7, 4, 8] h=8 moves: 18
[2, 3, 5, 1, 4, 6, 7, 0, 8] h=7 moves: 19
[2, 3, 5, 1, 4, 6, 7, 8, 0] h=6 moves: 20
[2, 3, 5, 1, 4, 0, 7, 8, 6] h=7 moves: 21
[2, 3, 0, 1, 4, 5, 7, 8, 6] h=6 moves: 22
[2, 0, 3, 1, 4, 5, 7, 8, 6]

In [8]:
print_succ([1,2,3,4,5,0,6,7,8])

[1, 2, 0, 4, 5, 3, 6, 7, 8] h= 6
[1, 2, 3, 4, 0, 5, 6, 7, 8] h= 6
[1, 2, 3, 4, 5, 8, 6, 7, 0] h= 6
